In [12]:

import jsonlines
import json
from db import DB
from generate_data import getData_html2
from rag import read_toml_config
def read_jsonl(path):
    content = []
    with jsonlines.open(path, "r") as json_file:
        for obj in json_file.iter(type=dict, skip_invalid=True):
            content.append(obj)
    return content



question = read_jsonl("../question.jsonl") 
answers = read_jsonl("test6.jsonl")
new_answers = []
for index_x,q in enumerate(question):
    for index_y,ans in enumerate(answers):
        # if ans["query"] == q["query"]:
        if q["id"] == ans["id"]:
            # if ans["query"]["query"] != q["query"]:
                # print(ans["query"],q["query"])
            print(q["id"])
            # if q["id"] == 36:
                # print(ans["answer"])
            try:
                new_answers.append({"id":q["id"],"query":q["query"],"answer":ans["answer"],"information":ans["information"]})
            except:
                print(q["id"],ans["answer"])
print(len(new_answers))
with open("answer8.jsonl","w") as f:
    for ans in new_answers:
        json.dump(ans,f,ensure_ascii=False)
        f.write('\n')


# config = read_toml_config("config.toml")
# answers = read_jsonl("answer6.jsonl")
# query = [answer["query"] for answer in answers]
# source_information = getData_html2(config["dir_path"])
# database = DB(config["embedding_path"],source_information,"vector_index.faiss")
# information = database.get_information_by_index(query=query,k=config["top_k"])
# with_information = []
# for index, answer in enumerate(answers):
#     if answer["query"] == query[index]:
#         print(answer["id"])
#         with_information.append({"id":answer["id"],"query":answer["query"],"information":information[index],"answer":answer["answer"]})
# with open("with_information_answer6.jsonl","w") as f:
#     for ans in with_information:
#         json.dump(ans,f,ensure_ascii=False)
#         f.write('\n')







1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
103


In [ ]:
with open("with_information.jsonl","w") as f:
    for ans in with_information:
        json.dump(ans,f,ensure_ascii=False)
        f.write('\n')

In [ ]:
from typing import List
class Tree:
    def __init__(self,name:str="",information:str="") -> None:
        self.name:str = name
        self.information:str = information
        self.sons:List[Tree] = []
    def append(self,tree:"Tree"):
        self.sons.append(tree)
def prase_node(path:str) -> Tree:
    """
    @Param path nodes下的一个txt文件的路径
    """
    tree = Tree()
    log = False
    try:
        with open(path,"r",encoding="utf-8") as f:
            for index,line in enumerate(f):
                if index == 0:
                    tree.name = line.strip(" ").strip("\n").strip(" ")
                    # print("name: ",tree.name,len(tree.name))
                else:
                    if "子主题" in line:
                        log = True
                if log:
                    if "*" in line:
                        tree.append(Tree(line.strip("\n").strip(" ").strip("*").strip(" ")))
                        # print("sons: ",line.strip("\n").strip(" ").strip("*"),len(line.strip("\n").strip(" ").strip("*").strip(" ")))

    except:
        with open(path,"r",encoding="gbk") as f:
             for index,line in enumerate(f):
                if index == 0:
                    tree.name = line.strip(" ").strip("\n")
                else:
                    if "子主题" in line:
                        log = True
                if log:
                    if "*" in line:
                        tree.append(Tree(line.strip("\n").strip(" ").strip("*")))
    return tree
def getTreeSonsList(tree:Tree,layer:int) -> List[Tree]:
    sons_list = []
    if layer == 0 and tree.sons == []:
        return sons_list
    for t in tree.sons:
        if t.sons == []:
            sons_list.append(t)
        else:
            sons = getTreeSonsList(t,layer+1)
            for i in sons:
                sons_list.append(i)

    return sons_list
def show_tree(tree:List[Tree]):
    # print(tree.name)
    if not tree:  # 如果列表为空，则不进行递归调用
        return
    ls = []
    for sons in tree:
        print(sons.name,end=" ")
        if sons.sons != []:
            for s in sons.sons:
                ls.append(s)
    print()
    show_tree(ls)
        
def concat_tree(tree_list:List[Tree]) -> List[Tree]:
    """
    合并一个List[Tree], 找出一个树的所有子节点的列表，遍历这个列表，遍历整个树的list
    如果发现能够合并，就直接合并，然后修改子节点的列表，重新遍历整个list，直到没有改变
    被合并的tree被标记为合并的树的序号
    @Param tree_list 需要合并的tree
    """
    change = False
    delete_list = [-1]*len(tree_list)
    for index_x,tree_x in enumerate(tree_list):
        if tree_x is None:
            continue
        # show_tree([tree_x])
        sons_list = getTreeSonsList(tree_x,0)
        # print("index_x",index_x)
        if sons_list == []:
            for index_y,tree_y in enumerate(tree_list):
                # print("index_y",index_y)
                if index_x == index_y:
                    continue
                for sons in sons_list:
                    if tree_y.name == sons.name:
                        if delete_list[index_x] == -1:
                            change = True
                            sons.sons = tree_y.sons
                            delete_list[index_y] = index_x
                            print(f"delete_list[{index_y}] = {index_x}") 
    # delete tree
    for index, num in enumerate(delete_list):
        if num != -1:
            tree_list.remove(tree_list[num])
    if change:
        return concat_tree(tree_list)
    return tree_list
                                           

In [ ]:
prase_node("/home/root1/wcc/AIOPS/aiops24-RAG-demo/demo/data/director/nodes/告警SNMP北向接口规范.txt")
dir_paths = []
import os
for root,dir,files in os.walk("/home/root1/wcc/AIOPS/aiops24-RAG-demo/demo/data/director/nodes"):
    print(root)# 根路径
    print(dir)# 子文件夹
    print(files)# 文件名
    for file in files:
        dir_paths.append(os.path.join(root,file))
tree_list = [prase_node(dir_path) for dir_path in dir_paths]
print(len(tree_list))
# for tree in tree_list:
#     print(tree.name)
concat = concat_tree(tree_list)
print(len(concat))
for con in concat:
    print(con.name)


In [ ]:
question = read_jsonl("../question.jsonl") 
answers = read_jsonl("../aiops-2024-submit/test2.jsonl")
new_answers = []
for q in question:
    for ans in answers:
        if ans["query"] == q["query"]:
            print(q["id"])
            new_answers.append({"id":q["id"],"query":q["query"],"answer":ans["answer"]})
with open("answer3.jsonl","w") as f:
    for ans in new_answers:
        json.dump(ans,f,ensure_ascii=False)
        f.write('\n')

In [ ]:
a = 0
length = 0
num_line = 0
with open("test.txt","r") as f:
    for line in f:
        num_line += 1
        if len(line) > 0:
            a += 1
            length += len(line)
print("非空行:" , a)
print("平均长度：", length/float(num_line))
